In [ ]:
import sys
path= '/home/javierz/PythonEnv/Python/Classes'
sys.path.append('/home/javierz/PythonEnv/Python/Classes')
from PS import PS

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import keras_tuner as kt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import seaborn as sns

In [ ]:
df_ids = pd.read_csv('../MIMIC III/v3.0/hadms_POP_S_SS_clean.csv')
vhadm = df_ids.loc[:,'hadm_id']

In [ ]:
for hadm in vhadm:
    print(hadm)
    
    df_1 = pd.read_csv('Data2/Resultados/MIMIC_Classif_POP_S_SS/'+str(hadm)+'.csv',index_col=0)

    C0_1 = df_1['C_0'].to_numpy()
    C1_1 = df_1['C_1'].to_numpy()
    C2_1 = df_1['C_2'].to_numpy()
    # vID_1 = df_1['ID']
    n_1 = len(df_1['C_0'])
    c_g_1 = np.zeros(n_1)

    for j in range(n_1):
        if(C0_1[j] > C1_1[j] and C0_1[j] > C2_1[j]):
            c_g_1[j] = 0
        if(C1_1[j] > C0_1[j] and C1_1[j] > C2_1[j]):
            c_g_1[j] = 1
        if(C2_1[j] > C0_1[j] and C2_1[j] > C1_1[j]):
            c_g_1[j] = 2
    
    df_new_1 = pd.DataFrame({'c_g':c_g_1})
    df_new_1['c_g'] = df_new_1['c_g'].astype('category')

    vCG,nW = PS.Trimmer(c_g_1,10) 
    
    vWin = []
    vID = []
    clase_0 = []
    clase_1 = []
    clase_2 = []

    a=0
    for i in range(nW):
        vWin.append(i)
        vID.append(hadm)
        val = df_new_1['c_g'][0+(i*10):10+(i*10)].value_counts(sort=False)
        index = df_new_1['c_g'][0+(i*10):10+(i*10)].value_counts(sort=False).index.tolist()
        
        if(len(index)<3):
            if(index == [0,1]):
                clase_0.append(np.round(val[0]*10,2))
                clase_1.append(np.round(val[1]*10,2))
                clase_2.append(0)
            if(index == [0,2]):
                clase_0.append(np.round(val[0]*10,2))
                clase_1.append(0)
                clase_2.append(np.round(val[1]*10,2))
            if(index == [1,2]):
                clase_0.append(0)
                clase_1.append(np.round(val[0]*10,2))
                clase_2.append(np.round(val[1]*10,2))
            if(index == [0]):
                clase_0.append(np.round(val[0]*10,2))
                clase_1.append(0)
                clase_2.append(0)
            if(index == [1]):
                clase_0.append(0)
                clase_1.append(np.round(val[0]*10,2))
                clase_2.append(0)
            if(index == [2]):
                clase_0.append(0)
                clase_1.append(0)
                clase_2.append(np.round(val[0]*10,2))
               

        else:            
            clase_0.append(np.round(val[0]*10,2))
            clase_1.append(np.round(val[1]*10,2))
            clase_2.append(np.round(val[2]*10,2))
        
    df_out = pd.DataFrame({'hadm_id':vID,'Window':vWin,'ANS_Basal_1':clase_0,'ANS_Symp_1':clase_1,'ANS_Parasymp_1':clase_2})

    df_out.to_csv('Data2/Resultados/MIMIC_ANS_POP_S_SS/'+str(hadm)+'.csv')  
    # break


print('rdy')     